In [1]:
from source.utils.npz_utils import get_smiles_and_filepaths_from_valid_npz
from source.utils.data_splitting_utils import scaffold_splitter

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/home/nobilm@usi.ch/miniconda3/envs/rdkit_pyg_geqm9_clone/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [3]:
dir = '/storage_common/nobilm/pretrain_paper/guacamol/5k'
smiles, filepaths = get_smiles_and_filepaths_from_valid_npz(dir)

In [4]:
train_smiles,val_smiles,test_smiles=scaffold_splitter(smile_list=smiles, save_dir='tmp', filepath_list=filepaths)

In [8]:
# len(train_smiles), train_smiles

In [9]:
# len(val_smiles), val_smiles

In [10]:
# len(test_smiles), test_smiles

In [ ]:
# TODO
# 1) create folders train/val/test_smiles
# 2) copy from target dir to created dirs wrt filepath